In [ ]:
import pandas as pd

from pathlib import Path

In [ ]:
def set_df_month(df_uf, customer_file):
	df = pd.read_csv(customer_file)
	df['ddd'].astype(int)
	
	df_merged = df.merge(df_uf, how = 'inner', on = 'ddd')
	df_merged = df_merged.iloc[:, :5]
	df_grouped = df_merged.groupby(['month', 'uf', 'code'], as_index = False)['qty'].sum()

	return df_grouped

In [ ]:
def set_weight(qty, min_qty, max_qty):
	if min_qty == max_qty:
		return 0
	
	return (qty - min_qty) / (max_qty - min_qty)

In [ ]:
def set_dataframe():
	df_uf = pd.read_csv('sources/uf-ddd.csv', sep = ';')
	path = Path('sources/customer-data')
	dataframes = []

	for customer_file in path.glob('*.csv'):
		df_month = set_df_month(df_uf, customer_file)
		dataframes.append(df_month)

	df_concat = pd.concat(dataframes, ignore_index = True)
	min_qty = df_concat['qty'].min()
	max_qty = df_concat['qty'].max()

	df_concat['weight'] = df_concat['qty'].apply(lambda x: set_weight(x, min_qty, max_qty))

	return df_concat

In [ ]:
df = set_dataframe()

In [ ]:
df.to_csv('sources/results.csv', index = False)